<a href="https://colab.research.google.com/github/Aliksm-DataScientist/RAG_With_Chat_History/blob/main/Chat_History_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain-community langchain-core

In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader

In [ ]:
loader = CSVLoader(file_path="/content/drive/MyDrive/Colab Notebooks/Dataset/my_portfolio.csv")

data = loader.load()

print(data)

[Document(metadata={'source': '/content/drive/MyDrive/Colab Notebooks/Dataset/my_portfolio.csv', 'row': 0}, page_content='Techstack: React, Node.js, MongoDB\nLinks: https://example.com/react-portfolio'), Document(metadata={'source': '/content/drive/MyDrive/Colab Notebooks/Dataset/my_portfolio.csv', 'row': 1}, page_content='Techstack: Angular,.NET, SQL Server\nLinks: https://example.com/angular-portfolio'), Document(metadata={'source': '/content/drive/MyDrive/Colab Notebooks/Dataset/my_portfolio.csv', 'row': 2}, page_content='Techstack: Vue.js, Ruby on Rails, PostgreSQL\nLinks: https://example.com/vue-portfolio'), Document(metadata={'source': '/content/drive/MyDrive/Colab Notebooks/Dataset/my_portfolio.csv', 'row': 3}, page_content='Techstack: Python, Django, MySQL\nLinks: https://example.com/python-portfolio'), Document(metadata={'source': '/content/drive/MyDrive/Colab Notebooks/Dataset/my_portfolio.csv', 'row': 4}, page_content='Techstack: Java, Spring Boot, Oracle\nLinks: https://exa

In [ ]:
!pip install langchain-groq

In [ ]:
from langchain_groq import ChatGroq

In [ ]:
llm = ChatGroq(
    model="mixtral-8x7b-32768",
    groq_api_key='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',
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)
# In place of llm we can pass direct model as like this=> model = ChatGoogleGenerativeAI(model="gemini-1.0-pro",convert_system_message_to_human=True)

In [ ]:
!pip install langchain_huggingface

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [ ]:
from langchain_community.vectorstores import FAISS

In [ ]:
!pip install faiss-cpu

In [ ]:
db = FAISS.from_documents(documents=data, embedding=embeddings)

In [ ]:
retriever = db.as_retriever()

In [44]:
result = retriever.invoke("Please do provide me the links related to the React.js")
print('Links Bro', result)

Links Bro [Document(id='7cb54ff7-2eae-4bcf-b703-fec6db4d9c41', metadata={'source': '/content/drive/MyDrive/Colab Notebooks/Dataset/my_portfolio.csv', 'row': 0}, page_content='Techstack: React, Node.js, MongoDB\nLinks: https://example.com/react-portfolio'), Document(id='423ffc34-eb9d-499a-aa92-46a99c04cf50', metadata={'source': '/content/drive/MyDrive/Colab Notebooks/Dataset/my_portfolio.csv', 'row': 8}, page_content='Techstack: React Native, Node.js, MongoDB\nLinks: https://example.com/react-native-portfolio'), Document(id='e3f6abc7-fe86-4d81-98f5-33e0d59d09df', metadata={'source': '/content/drive/MyDrive/Colab Notebooks/Dataset/my_portfolio.csv', 'row': 17}, page_content='Techstack: Full-stack, JavaScript, Express.js\nLinks: https://example.com/full-stack-js-portfolio'), Document(id='70be59bb-b366-4f28-ae42-0a9f1880b906', metadata={'source': '/content/drive/MyDrive/Colab Notebooks/Dataset/my_portfolio.csv', 'row': 16}, page_content='Techstack: Frontend, TypeScript, Angular\nLinks: htt

In [57]:
for doc in result:
    if "React, Node.js, MongoDB" in doc.page_content:
        print(doc.page_content)

Techstack: React, Node.js, MongoDB
Links: https://example.com/react-portfolio


In [45]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer the question "
    "If you don't know the answer, say that you don't know."
    "Use three sentences maximum and keep the answer concise."
    "\n\n"
    "{context}"
)

In [46]:
from langchain_core.prompts import ChatPromptTemplate

In [47]:
chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [48]:
from langchain.chains.combine_documents import create_stuff_documents_chain

In [49]:
question_answering_chain=create_stuff_documents_chain(llm, chat_prompt)

In [50]:
from langchain.chains import create_retrieval_chain

In [51]:
rag_chain = create_retrieval_chain(retriever, question_answering_chain)

In [52]:
result = retriever.invoke("Please do provide me the links related to the javascript.js")

In [54]:
result

[Document(id='e3f6abc7-fe86-4d81-98f5-33e0d59d09df', metadata={'source': '/content/drive/MyDrive/Colab Notebooks/Dataset/my_portfolio.csv', 'row': 17}, page_content='Techstack: Full-stack, JavaScript, Express.js\nLinks: https://example.com/full-stack-js-portfolio'),
 Document(id='70be59bb-b366-4f28-ae42-0a9f1880b906', metadata={'source': '/content/drive/MyDrive/Colab Notebooks/Dataset/my_portfolio.csv', 'row': 16}, page_content='Techstack: Frontend, TypeScript, Angular\nLinks: https://example.com/typescript-frontend-portfolio'),
 Document(id='09e1b46c-481c-46c2-ae83-0df44f2396b8', metadata={'source': '/content/drive/MyDrive/Colab Notebooks/Dataset/my_portfolio.csv', 'row': 1}, page_content='Techstack: Angular,.NET, SQL Server\nLinks: https://example.com/angular-portfolio'),
 Document(id='7cb54ff7-2eae-4bcf-b703-fec6db4d9c41', metadata={'source': '/content/drive/MyDrive/Colab Notebooks/Dataset/my_portfolio.csv', 'row': 0}, page_content='Techstack: React, Node.js, MongoDB\nLinks: https:/

In [58]:
from langchain.chains import create_history_aware_retriever

In [59]:
retriever_prompt = (
    "Given a chat history and the latest user question which might reference context in the chat history,"
    "formulate a standalone question which can be understood without the chat history."
    "Do NOT answer the question, just reformulate it if needed and otherwise return it as is."
)

In [62]:
from langchain_core.prompts import MessagesPlaceholder

In [63]:
contextualize_q_prompt  = ChatPromptTemplate.from_messages(
    [
        ("system", retriever_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),


     ]
)

In [65]:
history_aware_retriever = create_history_aware_retriever(llm,retriever,contextualize_q_prompt)

In [66]:
from langchain.chains import create_retrieval_chain

In [67]:
from langchain.chains.combine_documents import create_stuff_documents_chain

In [68]:

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [69]:
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

In [70]:
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [71]:
from langchain_core.messages import HumanMessage, AIMessage

In [72]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [73]:
store = {}

In [74]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [75]:
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [82]:
conversational_rag_chain.invoke(
    {"input": "Are there any python links?"},
    config={
        "configurable": {"session_id": "Gyan156"}
    },
)["answer"]

'Yes, there are two links that include Python in the techstack:\n1. <https://example.com/python-portfolio>\n2. <https://example.com/ml-python-portfolio>'

In [83]:
conversational_rag_chain.invoke(
    {"input": "Please provide me the React.js Links"},
    config={"configurable": {"session_id": "Gyan156"}},
)["answer"]

'Here are the two links related to React.js:\n1. <https://example.com/react-portfolio>\n2. <https://example.com/react-native-portfolio>'

In [84]:
store

{'Gyan156': InMemoryChatMessageHistory(messages=[HumanMessage(content='Are there any python links?', additional_kwargs={}, response_metadata={}), AIMessage(content='Yes, there are two links that include Python in the techstack:\n1. <https://example.com/python-portfolio>\n2. <https://example.com/ml-python-portfolio>', additional_kwargs={}, response_metadata={}), HumanMessage(content='Please provide me the React.js Links', additional_kwargs={}, response_metadata={}), AIMessage(content='Here are the two links related to React.js:\n1. <https://example.com/react-portfolio>\n2. <https://example.com/react-native-portfolio>', additional_kwargs={}, response_metadata={})])}

In [85]:
for message in store["Gyan156"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")

User: Are there any python links?

AI: Yes, there are two links that include Python in the techstack:
1. <https://example.com/python-portfolio>
2. <https://example.com/ml-python-portfolio>

User: Please provide me the React.js Links

AI: Here are the two links related to React.js:
1. <https://example.com/react-portfolio>
2. <https://example.com/react-native-portfolio>



In [80]:
del store['Gyan156']

In [81]:
store

{}